In [1]:
import os
import numpy as np
import networkx as nx
import pandas as pd
import copy
import random

#Pandapower Package
import pandapower as pp
import pandapower.networks as pn
import pandapower.plotting as plot

from pandapower.powerflow import LoadflowNotConverged
from pandapower.diagnostic import diagnostic
from pandapower.control import ConstControl
from pandapower.timeseries import DFData, OutputWriter, run_timeseries
from pandapower.pypower.makeYbus import makeYbus

import matplotlib.pyplot as plt
from scipy.stats import norm
import scipy.linalg
import scipy.sparse as sp
from tqdm import tqdm

# Prioritize and select the most impactful N-2 contingency components (lines or generators: conventional/ solar) 
#### Based on analytical insights, not random choice.

### Define the Function to check N-1 contingency criterion

In [2]:
# Define function to check whether the network meet the N-1 contingency criterion
def check_n_1_contingency(net):
    critical_elements = []
    
    # Backup original network
    original_net = copy.deepcopy(net)
    
    # Test line outages
    for line in net.line.index:
        net_copy = copy.deepcopy(original_net)
        net_copy.line.at[line, "in_service"] = False  # Remove one line
        try:
            pp.runpp(net_copy, algorithm="nr")
        except pp.powerflow.LoadflowNotConverged:
            critical_elements.append(f"Line {line} outage causes failure.")
            continue
        
        # Check for overloads
        if any(net_copy.res_line.loading_percent > 100):
            critical_elements.append(f"Line {line} outage causes overloads.")

    # Test generator outages
    for gen in net.gen.index:
        net_copy = copy.deepcopy(original_net)
        net_copy.gen.at[gen, "in_service"] = False  # Remove one generator
        try:
            pp.runpp(net_copy, algorithm="nr")
        except pp.powerflow.LoadflowNotConverged:
            critical_elements.append(f"Generator {gen} outage causes failure.")
            continue

        # Check for voltage violations
        if any((net_copy.res_bus.vm_pu < 0.95) | (net_copy.res_bus.vm_pu > 1.05)):
            critical_elements.append(f"Generator {gen} outage causes voltage issues.")

    return critical_elements

### Load the Network Testcase

In [3]:
# Load the 30-bus system
net = pn.case30()

### Built the Function to Reinforce the network to meet the criterion

In [4]:
# Function to Reinforce the network based on contingency violations
def reinforce_network(net):
    original_net = copy.deepcopy(net)
    overloaded_lines = []
    problematic_gens = []
   
    for line in net.line.index:
        net_copy = copy.deepcopy(original_net)
        net_copy.line.at[line, "in_service"] = False  # Simulate line outage
        try:
            pp.runpp(net_copy)
        except pp.powerflow.LoadflowNotConverged:
            overloaded_lines.append(line)
            continue
        if any(net_copy.res_line.loading_percent > 100):
            overloaded_lines.append(line)
    
    # Add parallel lines to overloaded lines
    for line in overloaded_lines:
        from_bus = net.line.loc[line, "from_bus"]
        to_bus = net.line.loc[line, "to_bus"]
        print(f"Adding parallel line between Bus {from_bus} and Bus {to_bus} to mitigate overload.")
        pp.create_line_from_parameters(net, from_bus=from_bus, to_bus=to_bus, 
                                       length_km=1.0, r_ohm_per_km=0.05, 
                                       x_ohm_per_km=0.1, c_nf_per_km=0, 
                                       max_i_ka=1.5)  # higher capacity
    
    # Identify problematic generators
    problematic_gens = []
    for gen in net.gen.index:
        net_copy = copy.deepcopy(original_net)
        net_copy.gen.at[gen, "in_service"] = False  # Simulate generator outage
        try:
            pp.runpp(net_copy)
        except pp.powerflow.LoadflowNotConverged:
            problematic_gens.append(gen)
            continue
        if any((net_copy.res_bus.vm_pu < 0.95) | (net_copy.res_bus.vm_pu > 1.05)):
            problematic_gens.append(gen)

    # Increase generator capacities to provide redundancy
    for gen in problematic_gens:
        net.gen.at[gen, "p_mw"] *= 1.2  # Increase power generation by 20%
        print(f"Increasing capacity of Generator {gen} to improve voltage stability.")

    return net

# Reinforce the network
net = reinforce_network(net)

# Run contingency check again
pp.runpp(net)
print("Reinforced system power flow successful.")

# Check if violations still exist
violations = check_n_1_contingency(net)
if violations:
    print("grid netwrok still has issues under N-1 contingency.")
    for v in violations:
        print(v)
else:
    print("The grid netwrok satisfies the N-1 contingency criterion!")

Adding parallel line between Bus 0 and Bus 1 to mitigate overload.
Adding parallel line between Bus 0 and Bus 2 to mitigate overload.
Adding parallel line between Bus 1 and Bus 3 to mitigate overload.
Adding parallel line between Bus 2 and Bus 3 to mitigate overload.
Adding parallel line between Bus 1 and Bus 4 to mitigate overload.
Adding parallel line between Bus 1 and Bus 5 to mitigate overload.
Adding parallel line between Bus 3 and Bus 5 to mitigate overload.
Adding parallel line between Bus 4 and Bus 6 to mitigate overload.
Adding parallel line between Bus 5 and Bus 6 to mitigate overload.
Adding parallel line between Bus 5 and Bus 7 to mitigate overload.
Adding parallel line between Bus 5 and Bus 8 to mitigate overload.
Adding parallel line between Bus 5 and Bus 9 to mitigate overload.
Adding parallel line between Bus 8 and Bus 10 to mitigate overload.
Adding parallel line between Bus 8 and Bus 9 to mitigate overload.
Adding parallel line between Bus 3 and Bus 11 to mitigate ove

### The PV bus (which contains the conventional generators) information

In [5]:
# View all generator (gen) information and Slack bus
gen_info = net.gen
print(gen_info)
print(net.ext_grid)

# Print the bus number where the generator is located
pv_bus_numbers = net.gen['bus'].values
print("Generator (PV) bus numbers:", pv_bus_numbers)

# Identify the 5 PV buses in the 30-bus system
pv_buses = [1, 21, 26, 22, 12]

   name  bus    p_mw  vm_pu  sn_mva  min_q_mvar  max_q_mvar  scaling  slack  \
0  None    1  73.164    1.0     NaN       -20.0        60.0      1.0  False   
1  None   21  25.908    1.0     NaN       -15.0        62.5      1.0  False   
2  None   26  32.292    1.0     NaN       -15.0        48.7      1.0  False   
3  None   22  19.200    1.0     NaN       -10.0        40.0      1.0  False   
4  None   12  37.000    1.0     NaN       -15.0        44.7      1.0  False   

   in_service  slack_weight  type  controllable  max_p_mw  min_p_mw  
0        True           0.0  None          True      80.0       0.0  
1        True           0.0  None          True      50.0       0.0  
2        True           0.0  None          True      55.0       0.0  
3        True           0.0  None          True      30.0       0.0  
4        True           0.0  None          True      40.0       0.0  
   name  bus  vm_pu  va_degree  slack_weight  in_service  max_p_mw  min_p_mw  \
0  None    0    1.0      

### load bus information

In [6]:
# View the bus information of the external power grid (Slack bus)
slack_bus_numbers = net.ext_grid['bus'].values
print("Slack bus numbers:", slack_bus_numbers)

# View load information
print(net.load)

# Calculate total active power and reactive power
total_p = net.load['p_mw'].sum()  # Total active power
total_q = net.load['q_mvar'].sum()  # Total reactive power

print(f"Total active power load of 30-bus system: {total_p} MW")
print(f"Total reactive power load of 30-bus system: {total_q} MVar")

Slack bus numbers: [0]
    name  bus  p_mw  q_mvar  const_z_percent  const_i_percent  sn_mva  \
0   None    1  21.7    12.7              0.0              0.0     NaN   
1   None    2   2.4     1.2              0.0              0.0     NaN   
2   None    3   7.6     1.6              0.0              0.0     NaN   
3   None    6  22.8    10.9              0.0              0.0     NaN   
4   None    7  30.0    30.0              0.0              0.0     NaN   
5   None    9   5.8     2.0              0.0              0.0     NaN   
6   None   11  11.2     7.5              0.0              0.0     NaN   
7   None   13   6.2     1.6              0.0              0.0     NaN   
8   None   14   8.2     2.5              0.0              0.0     NaN   
9   None   15   3.5     1.8              0.0              0.0     NaN   
10  None   16   9.0     5.8              0.0              0.0     NaN   
11  None   17   3.2     0.9              0.0              0.0     NaN   
12  None   18   9.5     3.4 

### Deactive some conventional generators

In [8]:
# Deactive specific generators(bus 21 22 12) # Make the solar-based generators more significant 
# Drop Around 82 MW Acive Power with the conventional generators / with 120mw capacity
net.gen.drop(net.gen[net.gen['bus'].isin([21,22,12])].index, inplace=True)

# Verify the remaining generators
print(net.gen)

   name  bus    p_mw  vm_pu  sn_mva  min_q_mvar  max_q_mvar  scaling  slack  \
0  None    1  73.164    1.0     NaN       -20.0        60.0      1.0  False   
2  None   26  32.292    1.0     NaN       -15.0        48.7      1.0  False   

   in_service  slack_weight  type  controllable  max_p_mw  min_p_mw  
0        True           0.0  None          True      80.0       0.0  
2        True           0.0  None          True      55.0       0.0  


In [9]:
print(net.sgen)

Empty DataFrame
Columns: [name, bus, p_mw, q_mvar, sn_mva, scaling, in_service, type, current_source]
Index: []


### Add the Static generators (Solar-Generation) 
### build the constant power factor control mode with reactive power on sgen

In [10]:
# Define Parameters
total_drop_conventional_capacity= 120  # MW capacity for deactiving the conventional generators
n = 8  # Number of added static（solar) generators as solar injection (Set this number between 5 and 10)

# Select buses (load buses) to add the solar generation injection with the static generators
# solar_sgen_buses= [3, 4, 6, 10, 12, 14, 16, 19]  # Buses with the solar-based generator
solar_sgen_buses= [1, 6, 7, 11, 14, 16, 20, 29]
assert len(solar_sgen_buses) == n, "Number of buses must match number of generators."
# Distribute total PV capacity equally per static generator
solar_installed_capacities_per_sgen = np.full(n, total_drop_conventional_capacity / n)

# Constant power factor 
power_factor = 0.95
q_mvar_per_sgen = solar_installed_capacities_per_sgen * np.tan(np.arccos(power_factor))

# Add static generators
for i, bus in enumerate(solar_sgen_buses):
    pp.create_sgen(net, 
                   bus=bus, 
                   p_mw=0, 
                   q_mvar=q_mvar_per_sgen[i], 
                   max_p_mw=solar_installed_capacities_per_sgen[i], 
                   name=f"Solar_StaticGen_{bus}", 
                   type="solar")

print(f"Added {n} solar-based static generators with {total_drop_conventional_capacity / n:.2f} MW Capcacity each.")
print(net.sgen)

Added 8 solar-based static generators with 15.00 MW Capcacity each.
                 name  bus  p_mw    q_mvar  sn_mva  scaling  in_service  \
0   Solar_StaticGen_1    1   0.0  4.930262     NaN      1.0        True   
1   Solar_StaticGen_6    6   0.0  4.930262     NaN      1.0        True   
2   Solar_StaticGen_7    7   0.0  4.930262     NaN      1.0        True   
3  Solar_StaticGen_11   11   0.0  4.930262     NaN      1.0        True   
4  Solar_StaticGen_14   14   0.0  4.930262     NaN      1.0        True   
5  Solar_StaticGen_16   16   0.0  4.930262     NaN      1.0        True   
6  Solar_StaticGen_20   20   0.0  4.930262     NaN      1.0        True   
7  Solar_StaticGen_29   29   0.0  4.930262     NaN      1.0        True   

    type  current_source  max_p_mw  
0  solar            True      15.0  
1  solar            True      15.0  
2  solar            True      15.0  
3  solar            True      15.0  
4  solar            True      15.0  
5  solar            True      15.0

### Bus Network information

In [11]:
print(" Basic Network Info:")
print(f"Number of Buses       : {len(net.bus)}")
print(f"Number of Lines       : {len(net.line)}")
print(f"Number of Generators  : {len(net.gen)}")
print(f"Number of Loads       : {len(net.load)}")

 Basic Network Info:
Number of Buses       : 30
Number of Lines       : 82
Number of Generators  : 2
Number of Loads       : 20


### Run Power Flow Analysis

In [12]:
pp.runpp(net)

### Identify Important Variables within the network

In [13]:
# Identify voltage-sensitive buses 
# Find buses with the largest deviation from 1.0 per unit (p.u.), which are more vulnerable to voltage instability.
voltage_deviation = abs(net.res_bus.vm_pu - 1.0)
top_voltage_buses = voltage_deviation.sort_values(ascending=False).head(5).index.tolist()
print(top_voltage_buses)

[18, 17, 20, 19, 16]


In [14]:
# Identify Heavily Loaded Lines 
top_lines_by_loading = net.res_line.loading_percent.sort_values(ascending=False).head(5).index.tolist()
print(top_lines_by_loading)

[41, 46, 55, 42, 44]


In [15]:
# Use the grah theory to find the critical buses
G = nx.Graph()
for _, row in net.line.iterrows():
    if row["in_service"]:
        G.add_edge(row["from_bus"], row["to_bus"], weight=row["length_km"])
centrality = nx.betweenness_centrality(G)
top_central_buses = sorted(centrality.items(), key=lambda x: x[1], reverse=True)
top_critical_buses = [bus for bus, _ in top_central_buses[:5]]

print(top_critical_buses)

[5, 9, 3, 11, 26]


In [16]:
combined_buses = set(top_voltage_buses) | set(top_critical_buses)
print(combined_buses)

{3, 5, 9, 11, 16, 17, 18, 19, 20, 26}


In [17]:
# Form a unified sensitive component group
sensitive_components =[]

# Add important lines
for line in top_lines_by_loading:
    sensitive_components.append(("line", line))

# Add lines connected to critical buses
for line in net.line.index:
    if (net.line.at[line, "from_bus"] in top_critical_buses or
        net.line.at[line, "to_bus"] in top_critical_buses):
        sensitive_components.append(("line", line))
        
# Add generators at voltage-sensitive or central buses (excluding dropped gens 21 and 22)
for gen in net.gen.index:
    bus = net.gen.at[gen, "bus"]
    if bus in combined_buses:
        sensitive_components.append(("gen", gen))      
        
# Add static generators (sgen) at voltage-sensitive or central buses
for sgen in net.sgen.index:
    bus = net.sgen.at[sgen, "bus"]
    if bus in combined_buses:
        sensitive_components.append(("sgen", sgen))
        
# Remove duplicates
sensitive_components = list(set(sensitive_components))
print(sensitive_components)

[('line', 49), ('line', 55), ('line', 58), ('line', 3), ('line', 67), ('line', 6), ('line', 9), ('line', 15), ('line', 76), ('line', 18), ('line', 24), ('line', 27), ('sgen', 5), ('line', 36), ('line', 42), ('line', 51), ('line', 57), ('line', 54), ('line', 5), ('line', 2), ('line', 8), ('line', 66), ('line', 11), ('line', 17), ('line', 78), ('line', 75), ('line', 14), ('line', 81), ('line', 26), ('line', 35), ('line', 41), ('sgen', 6), ('line', 44), ('line', 50), ('sgen', 3), ('line', 47), ('line', 56), ('line', 59), ('line', 65), ('line', 10), ('line', 68), ('line', 13), ('line', 77), ('line', 16), ('line', 25), ('gen', 2), ('line', 34), ('line', 37), ('line', 43), ('line', 40), ('line', 46), ('line', 52)]


In [18]:
# Separate line and generator entries
line_ids = [idx for typ, idx in sensitive_components if typ == "line"]
gen_ids = [idx for typ, idx in sensitive_components if typ == "gen"]
sgen_ids = [idx for typ, idx in sensitive_components if typ == "sgen"]

# Create dataframes with selected rows
df_sensitive_lines = net.line.loc[line_ids].copy()
df_sensitive_gens = net.gen.loc[gen_ids].copy()
df_sensitive_sgens = net.sgen.loc[gen_ids].copy()

# Add index as a column for clarity
df_sensitive_lines["line_id"] = df_sensitive_lines.index
df_sensitive_gens["gen_id"] = df_sensitive_gens.index
df_sensitive_gens["sgen_id"] = df_sensitive_sgens.index

# Rearrange columns to show ID first
df_sensitive_lines = df_sensitive_lines[["line_id"] + [col for col in df_sensitive_lines.columns if col != "line_id"]]
df_sensitive_gens = df_sensitive_gens[["gen_id"] + [col for col in df_sensitive_gens.columns if col != "gen_id"]]
df_sensitive_gens = df_sensitive_gens[["gen_id"] + [col for col in df_sensitive_gens.columns if col != "gen_id"]]

# Show
print("Sensitive Lines:")
display(df_sensitive_lines)

print("Sensitive Generators:")
display(df_sensitive_gens)

print("Sensitive static Generators:")
display(df_sensitive_sgens)

Sensitive Lines:


,line_id,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,max_loading_percent,geo
49,49,None,None,5,6,1.0,0.0500,0.1000,0.000000,0.0,1.500000,1.0,1,None,True,NaN,NaN
55,55,None,None,3,11,1.0,0.0500,0.1000,0.000000,0.0,1.500000,1.0,1,None,True,NaN,NaN
58,58,None,None,11,14,1.0,0.0500,0.1000,0.000000,0.0,1.500000,1.0,1,None,True,NaN,NaN
3,3,None,None,2,3,1.0,1.8225,7.2900,0.000000,0.0,0.555967,1.0,1,ol,True,100.0,NaN
67,67,None,None,9,20,1.0,0.0500,0.1000,0.000000,0.0,1.500000,1.0,1,None,True,NaN,NaN
6,6,None,None,3,5,1.0,1.8225,7.2900,0.000000,0.0,0.384900,1.0,1,ol,True,100.0,NaN
9,9,None,None,5,7,1.0,1.8225,7.2900,0.000000,0.0,0.136853,1.0,1,ol,True,100.0,NaN
15,15,None,None,11,12,1.0,0.0000,25.5150,0.000000,0.0,0.277983,1.0,1,ol,True,100.0,NaN
76,76,None,None,27,26,1.0,0.0500,0.1000,0.000000,0.0,1.500000,1.0,1,None,True,NaN,NaN
18,18,None,None,11,15,1.0,16.4025,36.4500,0.000000,0.0,0.136853,1.0,1,ol,True,100.0,NaN


Sensitive Generators:


,gen_id,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,in_service,slack_weight,type,controllable,max_p_mw,min_p_mw,sgen_id
2,2,None,26,32.292,1.0,NaN,-15.0,48.7,1.0,False,True,0.0,None,True,55.0,0.0,2


Sensitive static Generators:


,name,bus,p_mw,q_mvar,sn_mva,scaling,in_service,type,current_source,max_p_mw
2,Solar_StaticGen_7,7,0.0,4.930262,NaN,1.0,True,solar,True,15.0


In [19]:
df_sensitive = pd.DataFrame(sensitive_components, columns=["type", "index"])
df_sensitive.to_csv("sensitive_components_case3_527_p50.csv", index=False)

In [20]:
# Randomly select two unique components for N-2
np.random.seed(42)
selected_pair = random.sample(sensitive_components, 2)

print(" Selected N-2 components from sensitive group:")
for c in selected_pair:
    print(f"   → {c[0].capitalize()} {c[1]}")

 Selected N-2 components from sensitive group:
   → Line 55
   → Line 66
